In [ ]:
#This is for Know the satellites position with a specific time and date 

In [1]:
#Now I need to know the near 
import os, sys, time, datetime, re, gpstk
from ftplib import FTP
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def to_radians(array):
    rads = array*(np.pi /180.)
    return rads

def compute_distance(df,longitude, latitude):
    R = 6371e3 # earth radius metres, mean radius = 6371km
    n = len(df.Latitude)
    ArrayLat = pd.Series([latitude] * n)
    ArrayLon = pd.Series([longitude] * n)
    #print len(ArrayLat), type(ArrayLat), type(df.Latitude)
    phi1 = to_radians(df.Latitude) #degrees to radians
    phi2 = to_radians(ArrayLat)
    lambda1 = to_radians(df.Longitude)
    lambda2 = to_radians(ArrayLon)
    dphi = phi2 - phi1
    dlambda = lambda2 - lambda1
    a = np.sin(dphi/2.)**2 + (np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2.)**2 )
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1.-a))
    d = R * c
    return d

float_formatter = lambda x: "%.2f" % x

def to_skyplot(elevation, azimut):
    e = np.array(elevation)
    a = np.array(azimut)
    sx = ((90-e)/90) * np.sin (a * (np.pi/180))
    sy = ((90-e)/90) * np.cos (a * (np.pi/180))
    return (sx,sy)

In [3]:
#Thanks to https://github.com/twillis449
def gunzip_some_file(compressed_file,
                     uncompressed_file,
                     delete_file = 1):
    print 'uncompressing file 1 to file 2 ', compressed_file, uncompressed_file
# if file is already uncompressed, do nothing
    if compressed_file == uncompressed_file:
        return
    # make sure there is a file
    if not os.path.isfile(compressed_file):
        if(compressed_file[-1] == 'Z'):
            # try another form
            warnings.warn("No .Z form of compressed file '%s', trying .gz"%compressed_file)
            print "No .Z form of compressed file  trying .gz"
            new_compressed = compressed_file[:-1] + "gz"
            return gunzip_some_file(new_compressed, uncompressed_file,
                                    delete_file)
        raise Albus_RINEX.RINEX_Data_Barf("No such file '%s' to uncompress"%compressed_file)
    if compressed_file[-3:] != 'zip' and compressed_file[-1] != 'Z':
        # file is most likely uncompressed ...
        command = "mv  %s %s"%(compressed_file, uncompressed_file)
    elif (compressed_file[-3:] == 'zip'): # we have a trignet file
        command = "unzip -np %s '*d' > %s"%(compressed_file,uncompressed_file)
    else:   
        command = "gunzip -dc %s > %s"%(compressed_file,uncompressed_file)
    print 'gunzip executing ', command
    retcode = os.system(command)
    print 'gunzip returned ', retcode
# handle non-zero return code error from extraction 
    if(retcode):
        raise No_RINEX_File_Error("Could not run '%s'"%command)
    if(delete_file):
        try:
            os.remove(compressed_file)
        except:
            print 'failure to remove compressed file - probably not a compressed file'
    return


In [91]:
Sat_Time = "01:20:23"
Sat_Date = "10/01/2017"
Longitude_inGround = -73.1224429 
Latitude_inGround = 7.1388027


In [94]:
import datetime as dt
mytime = dt.datetime.strptime('0130','%H%M').time()
mydatetime = dt.datetime.combine(dt.date.today(), mytime)
print mydatetime

2017-02-02 01:30:00


In [96]:
#Parse String to Date
import datetime

time_sat = datetime.datetime.strptime(Sat_Time,'%H:%M:%S')
date_sat = datetime.datetime.strptime(Sat_Date,'%d/%m/%Y')

InputDate = datetime.datetime.combine(datetime.date(date_sat.year, date_sat.month, date_sat.day), datetime.time(time_sat.hour, time_sat.minute,time_sat.second))
print InputDate

2017-01-10 01:20:23


In [98]:
day_of_year = InputDate.timetuple().tm_yday
day_of_year

10

In [45]:
UNAVCO_list = pd.read_csv("UNAVCOlist.csv")
UNAVCO_list.columns

Index([u'Unnamed: 0', u'CODE', u'Longitude', u'Latitude'], dtype='object')

search the nearest station

In [9]:
#Universidad Industrial de Santander
Latitude_inGround,Longitude_inGround

(7.1388027, -73.1224429)

In [10]:
d = compute_distance(UNAVCO_list,Longitude_inGround,Latitude_inGround)
UNAVCO_list['Distance'] = d

In [11]:
UNAVCO_list.sort_values(['Distance'], ascending=[True], inplace=True)
UNAVCO_list

,Unnamed: 0,CODE,Longitude,Latitude,Distance
2632,2632,BUCM,-73.181700,7.116900,6977.043272
2673,2673,BUCM,-73.181700,7.116900,6977.043272
2901,2901,BUCM,-73.181700,7.116900,6977.043272
2690,2690,FLOR,-73.088600,7.041600,11435.389590
2668,2668,ARAT,-73.052200,6.703200,49053.488836
3048,3048,PLON,-72.649000,7.384000,58911.122695
25086,25086,MECE,-73.712000,7.107200,65144.656963
2728,2728,SOCO,-73.241700,6.462500,76345.500651
2871,2871,ABRE,-73.225900,8.086800,106027.388816
24149,24149,CUC1,-72.512800,7.932300,110911.258013


In [12]:
#search the nearest land stage
IDmin = UNAVCO_list.Distance.argmin()
S_Code = UNAVCO_list.CODE[IDmin]
S_Dist = UNAVCO_list.Distance[IDmin]
print S_Code,UNAVCO_list.Longitude[IDmin],UNAVCO_list.Latitude[IDmin],S_Dist

BUCM -73.1817 7.1169 6977.04327191


Now I have the Name of the Lan Station that I need.


In [13]:
UNAVCO_FTP = 'data-out.unavco.org'
NASA_FTP = 'cddis.gsfc.nasa.gov'

try:
    f = FTP(NASA_FTP)
    #f = FTP(UNAVCO_FTP)
    
except:
    pass
#print "Welcome:", f.getwelcome()
f.login()

#Now I wnat to look the folder that I need.
UNAVCO_FTP_path = '/pub/rinex/nav/' + str(InputDate.year) + '/' + str(day_of_year).zfill(3)
FTP_path ='/pub/gps/data/daily/'+ str(InputDate.year) + '/' + str(day_of_year).zfill(3) + '/' + str(InputDate.year)[2:] + 'n'

#[data-out.unavco.org] --> /pub/rinex/nav/2017/002
#[cddis.gsfc.nasa.gov] --> /pub/gps/data/daily/2017/001/17n --> 
#    FTP_path ='/pub/gps/data/daily/'+ str_year + '/' + str(day_of_year).zfill(3) + '/' + str_year[2:] + 'n'

print "Folder Needed:", f.cwd(FTP_path)
print "Current working directory:", f.pwd()

Folder Needed: 250 Directory successfully changed.
Current working directory: /pub/gps/data/daily/2017/010/17n


In [14]:
import os, sys
from ftplib import FTP 


LineData = 0
while (1):
    try:
        if(LineData>70):
            break
        IDmin = UNAVCO_list.Distance.argmin()
        NameStation = UNAVCO_list.CODE[IDmin]
        LineData += 1
        NameFile = NameStation.lower() + str(day_of_year).zfill(3) + '0.' + str(InputDate.year)[2:] + 'n.Z' 
        #For Now just GPS constallartion, I mean, .n file.'
        clear_output(wait=True)
        print("Try with station: %s" % (NameFile),"to " + str(float_formatter(UNAVCO_list.Distance[IDmin]/1000)) + " [Km]")
        print
        
        f.voidcmd("TYPE I")
        datasock, size = f.ntransfercmd("RETR "+ NameFile)
        bytes_so_far = 0
        fd = open(NameFile, 'wb')
        
        while 1:
            buf = datasock.recv(2048) 
            
            fd.write(buf)
            bytes_so_far += len(buf)
            clear_output(wait=True)
            print "Received",bytes_so_far, "of %d total bytes (%.1f%%)" % (size, 100 * bytes_so_far / float(size)),"bytes"
            sys.stdout.flush()
            if not buf:
                break
                 
            #if bytes_so_far == len(buf):
                #print 
                #break
        print("File %s" % (NameFile),"to " + str(float_formatter(UNAVCO_list.Distance[IDmin]/1000)) + " [Km]")
        fd.close()
        datasock.close()
        f.voidresp
        f.close()
        break
    except:
        UNAVCO_list = UNAVCO_list[UNAVCO_list.CODE != NameStation]

Received 40779 of 40779 total bytes (100.0%) bytes
('File medi0100.17n.Z', 'to 296.64 [Km]')


In [15]:
NameFile

'medi0100.17n.Z'

In [16]:
gunzip_some_file(NameFile,'navRINEX',0)

uncompressing file 1 to file 2  medi0100.17n.Z navRINEX
gunzip executing  gunzip -dc medi0100.17n.Z > navRINEX
gunzip returned  0


In [54]:
navfile = 'navRINEX'
navHeader, navData = gpstk.readRinex3Nav(navfile)
# setup ephemeris store to look for satellite positions
bcestore = gpstk.GPSEphemerisStore()
for navDataObj in navData:
    ephem = navDataObj.toGPSEphemeris()
    bcestore.addEphemeris(ephem)
bcestore.SearchNear()
navData.close()

In [99]:
import copy
systime = copy.copy(InputDate)
systime

datetime.datetime(2017, 1, 10, 1, 20, 23)

In [107]:
#Thx to https://github.com/scienceopen
# Read current time from my own date and time
systime = gpstk.SystemTime()

# Convert to 'CommonTime', the standard way to handle time at GPSTk
comtime = systime.toCommonTime()

# This is the typical way to handle civil time
civtime = gpstk.CivilTime(comtime)

# The YDSTime class is very useful for common GNSS tasks
ydstime = gpstk.YDSTime(comtime)

# This is a typical class to handle time in GPS system
gpstime = gpstk.GPSWeekSecond(comtime)

# Class to handle Modified Julian Date
mjd = gpstk.MJD(comtime)

print("Hello world!")
#print("   The current civil time is", civtime)
print("   The current year is", ydstime.year)
print("   The current day of year is", ydstime.doy)
print("   The current second of day is", ydstime.sod)
print("   The current full GPS week is", gpstime.week)
print("   The current short GPS week is", gpstime.getModWeek())
print("   The current day of GPS week is", gpstime.getDayOfWeek())
print("   The current second of GPS week is", gpstime.sow)
#print("   The current Modified Julian Date is", mjd)

Hello world!
('   The current year is', 2017)
('   The current day of year is', 34)
('   The current second of day is', 5239.693747)
('   The current full GPS week is', 1934)
('   The current short GPS week is', 910)
('   The current day of GPS week is', 5)
('   The current second of GPS week is', 437239.693747)


In [110]:
print bcestore.getInitialTime()
print bcestore.getFinalTime()
print "Input Date",InputDate
print "Time in UTS", systime
#print gpstime.setTimeSystem(gpstk.TimeSystem('GPS'))
gpstime1 = gpstk.TimeSystem("GPS")
print "Time in GPS",gpstime1

print gpstk.GPSWeek(systime)

print gpstime>bcestore.getInitialTime()
print gpstime<bcestore.getFinalTime()

2457633 43200000 0.000000000000000 GPS
2457765 07200000 0.000000000000000 GPS
Input Date 2017-01-10 01:20:23
Time in UTS 1486085239 693747 UTC
Time in GPS GPS


AttributeError: No constructor defined - class is abstract

In [67]:
for satID in range (1,32):
    eph   = bcestore.findEphemeris(satID, gpstime)
    svXvt = eph.svXvt(gpstime)
    elev = obsHeader.antennaPosition.elvAngle(svXvt.getPos())
    azim  = obsHeader.antennaPosition.azAngle(svXvt.getPos())


TypeError: in method 'GPSEphemerisStore_findEphemeris', argument 2 of type 'gpstk::SatID const &'

In [ ]:
obsHeader, obsData = gpstk.readRinex3Obs(obsfile)
#obsObject = obsData.next()
for obsObject in obsData: #round the data
    for satID, datumList in obsObject.obs.iteritems():

        eph   = bcestore.findEphemeris(satID, obsObject.time)
        svXvt = eph.svXvt(obsObject.time)
        elev = obsHeader.antennaPosition.elvAngle(svXvt.getPos())
        azim    = obsHeader.antennaPosition.azAngle(svXvt.getPos())

        if(satID.system != satID.systemGPS):
            print satID.system, satID.id
        #print satID.system, satID.id
        AllSat.append('PRN ' + str(satID.id))
        Elevations.append(elev)
        Azimuths.append(azim)